## PO-240: Redes Neurais

### Importando Bibliotecas

In [3]:
import os

from datetime import datetime
import pandas as pd
import seaborn as sn
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow.keras.models      as models
import tensorflow.keras.losses      as losses
import tensorflow.keras.layers      as layers
import tensorflow.keras.metrics     as metrics
import tensorflow.keras.optimizers  as optimizers
import tensorflow.keras.activations as activations

In [7]:
from tensorflow.python.client import device_lib 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(device_lib.list_local_devices())

Num GPUs Available:  0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2814943367962273157
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7009365674716889354
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12186294957598253953
physical_device_desc: "device: XLA_GPU device"
]


### Importando o Dataset

In [6]:
dataset_folder = 'KaggleDatasets/RAW/'
preprocess_folder = 'KaggleDatasets/PRE/'
filename = 'KaggleDatasets/PRE/preprocessado_small.csv'
dataset = pd.read_csv(filename)
dataset.head()


,date,input_0,input_1,input_2,input_3,input_4_1,input_5_1,input_6_1,input_7_1,input_8_1,...,output_7_6,output_8_6,output_9_6,output_10_6,output_11_6,output_12_6,output_13_6,output_14_6,output_15_6,output_16_6
0,1,7,0,212,61,1.319214,-0.206138,2.953064,1.556292,0.585759,...,-0.699538,-0.590900,-0.191157,1.654277,0.263587,-0.748686,-0.083200,-0.669203,-0.250413,-0.450770
1,1,7,1,212,61,0.015678,-0.206138,-0.055500,0.112412,2.777662,...,-0.500047,-0.590900,-0.191157,-0.050361,-0.129333,-0.668941,-0.083200,0.010553,-0.456349,-0.450770
2,1,7,2,212,61,-0.718217,-0.206138,-0.657213,0.112412,-0.510193,...,-0.699538,-0.590900,-0.191157,-0.391288,-0.746778,-0.908174,-0.083200,-0.669203,-0.662284,-0.450770
3,1,7,3,212,61,-0.802090,-0.206138,-0.657213,-0.609528,-0.510193,...,-0.699538,-0.590900,-0.191157,-0.220824,-0.915172,-0.987918,-0.083200,-0.669203,-0.662284,-0.450770
4,1,13,0,183,142,-0.007038,-0.271512,0.924559,1.409946,0.810025,...,-0.149904,0.733469,0.444222,0.691026,0.246365,0.313061,-0.190019,0.467357,-0.468351,0.300399


### Separando o Dataset

In [8]:
cols_input = []
cols_output = []

for col in dataset.columns:
    if col.startswith("input") or col.startswith('date'):
        cols_input.append(col)
    elif col.startswith("output"):
        cols_output.append(col)
    else:
        print("Unexpected column name:", col)
        continue

inputs  = dataset[cols_input]
outputs = dataset[cols_output]

In [10]:
print('Inputs')
inputs.head()

Inputs


,date,input_0,input_1,input_2,input_3,input_4_1,input_5_1,input_6_1,input_7_1,input_8_1,...,input_547_14,input_548_14,input_549_14,input_550_14,input_551_14,input_552_14,input_553_14,input_554_14,input_555_14,input_556_14
0,1,7,0,212,61,1.319214,-0.206138,2.953064,1.556292,0.585759,...,1.809860,1.347449,-0.294892,1.389491,0.455686,0.509034,-0.073706,1.852765,-0.276792,-0.531387
1,1,7,1,212,61,0.015678,-0.206138,-0.055500,0.112412,2.777662,...,0.525939,0.863640,-0.294892,-1.030991,0.362408,0.517674,-0.073706,0.489274,-0.983316,-0.531387
2,1,7,2,212,61,-0.718217,-0.206138,-0.657213,0.112412,-0.510193,...,-0.757983,1.347449,-0.294892,-0.741760,-1.522024,-1.313605,-0.073706,-1.082850,0.231675,-0.531387
3,1,7,3,212,61,-0.802090,-0.206138,-0.657213,-0.609528,-0.510193,...,-0.757983,-0.602449,-0.294892,0.085311,-0.155945,-2.021802,-0.073706,-1.082850,-0.983316,-0.531387
4,1,13,0,183,142,-0.007038,-0.271512,0.924559,1.409946,0.810025,...,0.863996,-0.614311,2.071813,0.468146,0.134050,1.159395,-0.172821,0.516235,0.367480,-0.592766


In [12]:
print('Outputs')
outputs.head()

Outputs


,output_1_0,output_2_0,output_3_0,output_4_0,output_5_0,output_6_0,output_7_0,output_8_0,output_9_0,output_10_0,...,output_7_6,output_8_6,output_9_6,output_10_6,output_11_6,output_12_6,output_13_6,output_14_6,output_15_6,output_16_6
0,-0.696061,0.881745,0.079447,-0.059651,-0.236408,0.996766,-0.300557,-0.590900,-0.191157,0.290567,...,-0.699538,-0.590900,-0.191157,1.654277,0.263587,-0.748686,-0.083200,-0.669203,-0.250413,-0.450770
1,1.350537,0.724799,-0.531665,-0.693425,-0.236408,0.267222,-0.699538,-0.590900,-0.191157,0.034871,...,-0.500047,-0.590900,-0.191157,-0.050361,-0.129333,-0.668941,-0.083200,0.010553,-0.456349,-0.450770
2,-0.696061,-0.766185,-0.837222,-0.693425,-0.236408,-0.219141,-0.699538,-0.590900,-0.191157,-0.817448,...,-0.699538,-0.590900,-0.191157,-0.391288,-0.746778,-0.908174,-0.083200,-0.669203,-0.662284,-0.450770
3,-0.013862,-0.923131,-0.990000,-0.851868,-0.236408,-0.705503,-0.699538,-0.590900,-0.191157,-0.902679,...,-0.699538,-0.590900,-0.191157,-0.220824,-0.915172,-0.987918,-0.083200,-0.669203,-0.662284,-0.450770
4,0.151276,0.316134,0.127676,-0.229432,1.202157,0.397220,-0.478358,-0.723711,0.230410,0.963149,...,-0.149904,0.733469,0.444222,0.691026,0.246365,0.313061,-0.190019,0.467357,-0.468351,0.300399


In [14]:
X_train, X_val, Y_train, Y_val = train_test_split(inputs, outputs, test_size=0.30, random_state=57) 
# Sung, sung, háa, pbaet never gets old

print('# of training images:', X_train.shape)
print('# of cross-validation images:', X_val.shape)

# of training images: (9912, 7747)
# of cross-validation images: (4248, 7747)


### Criando o Modelo

In [16]:
alpha = 0.01
dropout_rate = 0.5

input_shape = X_train.shape[1]
output_shape = Y_train.shape[1]

In [18]:
def basic_model():
    input_layer = layers.Input(shape=(input_shape,))
    x = layers.Dense(1024, activation=activations.linear)(input_layer)
    x = layers.LeakyReLU(alpha)(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(1024, activation=activations.linear)(x)
    x = layers.LeakyReLU(alpha)(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(1024, activation=activations.linear)(x)
    x = layers.LeakyReLU(alpha)(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(1024, activation=activations.linear)(x)
    x = layers.LeakyReLU(alpha)(x)
    x = layers.Dropout(dropout_rate)(x)
    output_layer = layers.Dense(output_shape, activation=activations.linear)(x)
    return models.Model(inputs=input_layer, outputs=output_layer)

def convolutional_model():
    input_layer = layers.Input(shape=(7747,))
    # print(input_layer.shape)

    general_input  = input_layer[:,0:5]
    # print(general_input.shape)

    temporal_input = input_layer[:,5:]
    # print(temporal_input.shape)

    temporal_input = tf.reshape(temporal_input, [-1, 553,14,1])
    # print(temporal_input.shape)

    print(temporal_input.shape)
    x = layers.Conv1D(124, 5, activation='relu')(temporal_input)
    print(x.shape)
    x = layers.MaxPool2D(pool_size=(54,10))(x)
    print(x.shape)
    x = layers.Flatten()(x)
    print(x.shape)

    x = layers.Concatenate(axis=1)([general_input, x])
    x = layers.Dense(16*1024, activation=activations.linear)(x)
    x = layers.LeakyReLU(alpha)(x)
    x = layers.Dense(1024, activation=activations.linear)(x)
    x = layers.LeakyReLU(alpha)(x)
    output_layer = layers.Dense(output_shape, activation=activations.linear)(x)
    return models.Model(inputs=input_layer, outputs=output_layer)

In [20]:
# model = basic_model()
model = convolutional_model()

opt = optimizers.Adam()
model.compile(
    optimizer=opt, 
    loss=losses.mean_squared_error, 
    metrics=[metrics.mean_squared_error])
model.summary()

(None, 553, 14, 1)
(None, 553, 10, 124)
(None, 10, 1, 124)
(None, 1240)
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 7747)]       0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_3 (Te [(None, 7742)]       0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Reshape_1 (TensorFl [(None, 553, 14, 1)] 0           tf_op_layer_strided_slice_3[0][0]
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 553, 10, 124) 744         tf_op_layer_Reshape_1[0][0]      
_______________

### Treinamento

In [ ]:
batch_size = 1024

history = model.fit(
    X_train,
    Y_train,
    batch_size=batch_size,
    epochs=100,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_val, Y_val),
    validation_batch_size=batch_size,
)


In [ ]:
fig_format = 'png'

# Plotting cost function convergence
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Cost Function Convergence')
plt.legend(['Treinamento', 'Validação'])
plt.grid()
plt.savefig('convergence_imitation' + '.' + fig_format, format=fig_format)

In [ ]:
model.save_weights(filepath='./KaggleDatasets/Weights/final_model.hdf5')

In [ ]:
load_model(model_name + '.hdf5')